In [1]:
import tensorflow as tf
import numpy as np
import sys
sys.path.append(r'.\Pycharm')
from Captcha import gen_captcha,numbers

def text2vec(text):#text变为one hot coding
    vector = np.zeros(40)
    for i in range(len(text)):
        idx = i*10+ord(text[i])-48#ACSII转int
        vector[idx] = 1
    return vector

def gen_batch(size=100):#生成一批数据
    batch_x = np.zeros((size,9600),np.float64)
    batch_y = np.zeros((size,40))
    for i in range(size):
        text,image,data = gen_captcha()
        batch_x[i,:] = np.mean(image,-1).flatten()/255#rgb转为灰度值，再归一化
        batch_y[i,:] = text2vec(text)#text变为one hot coding
    return batch_x,batch_y
    

 
       
with tf.Session() as sess:
    #原始图
    '''text,image,data = gen_captcha()
    image0 = tf.reshape(image,shape=[-1,60,160,3])
    summary_op = tf.summary.image('image0',image0)
    writer = tf.summary.FileWriter('./log/demo15')
    merged = tf.summary.merge_all()
    summary = sess.run(merged)
    writer.add_summary(summary)'''
    
    #转为灰度图
    '''x1,y1 = gen_batch(1)
    image1 = tf.reshape(x1,shape=[-1,60,160,1])
    tf.summary.image('image1',image1)
    writer = tf.summary.FileWriter('./log/demo15/image-gray')
    merged = tf.summary.merge_all()
    summary = sess.run(merged)
    writer.add_summary(summary)'''
    
    #生成数据
    x1,y1 = gen_batch(1)
    x = tf.placeholder(tf.float32,[None,60*160])#不然的话类型不匹配
    image1 = tf.reshape(x,shape=[-1,60,160,1])
    tf.summary.image('image1',image1)
    
    #卷积并激活
    kernel = tf.Variable(tf.random_normal([3,3,1,32]))#（卷积核尺寸（奇数），图片维度，要达到的维度）
    b = tf.Variable(tf.random_normal([32]))
    image2 = tf.nn.conv2d(image1,kernel,strides=[1,1,1,1],padding='SAME')#strides:步长
    image2 = tf.nn.relu(tf.nn.bias_add(image2,b))#对image2 偏置并且relu激活
    #tf.summary.image('image2',image2)
    #池化
    image2 = tf.nn.max_pool(image2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')#（图片，核,步长，padding）其中核（批次通道，高度，宽度，颜色）
    image2 = tf.reshape(image2,shape=[-1,30,80,1])#因为步长为2，所以图片长宽会缩小2倍
    tf.summary.image('image2',image2)
    
    tf.global_variables_initializer().run()
    writer = tf.summary.FileWriter('./log/demo16/image-pool32')
    merged = tf.summary.merge_all()
    summary = sess.run(merged,feed_dict={x:x1})
    writer.add_summary(summary)

E:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
